In [10]:
import pandas as pd
import numpy as np
from math import ceil
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

In [12]:
######## Espaço para deep learning ########

## Dataset 

In [22]:
dataset = pd.read_csv('HAM10000_metadata.csv')

#Categorização do atributo 'localization'
data_localization = pd.get_dummies(dataset.localization)
dataset = pd.concat([dataset, data_localization], axis=1)

#Categorização do atributo 'sex'
data_sex = pd.get_dummies(dataset.sex)
dataset = pd.concat([dataset, data_sex], axis=1)

dataset = dataset.drop(['sex', 'localization', 'female'], axis = 1)

dataset.head()

,lesion_id,image_id,dx,dx_type,age,abdomen,acral,back,chest,ear,...,genital,hand,lower extremity,neck,scalp,trunk,unknown,upper extremity,male,unknown
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


## Regra da Pirâmide Geométrica

Implementação da Regra da Pirâmide Geométrica para determinação da quantidade de Neurônios Ocultos

        Nh = α·√(Ni·No) ; Nh = Número de Neurônios Ocultos
                          Ni = Número de Neurônios de Entrada
                          No = Número de Neurônios de Saída
                          α  = Constante

In [2]:
def piramide_geometrica(ni, no, alfa):
    nh = alfa*((ni*no)**(1/2))
    return ceil(nh)

##  Distribuição dos Neurônios em duas Camadas Ocultas

Função para gerar todas as possíveis 2-tuplas que representam o número de neurônios distribuídos por duas camadas ocultas de uma RNA do tipo MLP, dado o número de neurônios ocultos obtidos previamente pela Regra da Pirâmide Geométrica.

In [3]:
def hidden_layers(layers, nh):
    for i in range(1, nh):
        neurons_layers = (i, nh-i)
        layers.append(neurons_layers)
    return layers

### Criação de Lista de Camadas Ocultas a Partir da Regra da Pirâmide Geométrica

In [6]:
num_in = ?
num_out = 7
alpha = [0.5, 1, 2, 3]
layers = []

In [7]:
for i in range(len(alpha)):
    nh = piramide_geometrica(num_in, num_out, alpha[i])
    print('Para α = %.1f, Nh = %d'%(alpha[i],nh))
    hidden_layers(layers, nh)#insere cada possibilidade de camadas ocultas, dado o numero de neurônios, na lista 'layers'
    
print()
print('Distribuições de Camadas Ocultas:\n')
for i in layers:
    print(i)

Para α = 0.5, Nh = 4
Para α = 1.0, Nh = 7
Para α = 2.0, Nh = 14
Para α = 3.0, Nh = 21

Distribuições de Camadas Ocultas:

(1, 3)
(2, 2)
(3, 1)
(1, 6)
(2, 5)
(3, 4)
(4, 3)
(5, 2)
(6, 1)
(1, 13)
(2, 12)
(3, 11)
(4, 10)
(5, 9)
(6, 8)
(7, 7)
(8, 6)
(9, 5)
(10, 4)
(11, 3)
(12, 2)
(13, 1)
(1, 20)
(2, 19)
(3, 18)
(4, 17)
(5, 16)
(6, 15)
(7, 14)
(8, 13)
(9, 12)
(10, 11)
(11, 10)
(12, 9)
(13, 8)
(14, 7)
(15, 6)
(16, 5)
(17, 4)
(18, 3)
(19, 2)
(20, 1)


## Busca em Grade

In [9]:
parameters = {'solver': ['lbfgs', 'adam', 'sgd'], 
              'activation': ['identity', 'logistic', 'tanh', 'relu'],
              'hidden_layer_sizes': layers,
              'max_iter':[1000, 2000, 5000],
              'learning_rate_init':[0.005, 0.0005, 0.0001],
              'learning_rate': ['adaptive', 'constant']}

gs = GridSearchCV(MLPClassifier(), 
                  parameters, 
                  cv=3, 
                  scoring='accuracy')

In [8]:
x = dataset.drop(['dx', 'lesion_id', 'image_id', 'dx_type'], axis = 1) #Atributos preditores
y = dataset.dx #Atributo Alvo

### Treinamento 

Treinamento de todas as combinações de RNAs definidas no GridSearchCV( )

In [ ]:
gs.fit(x, y)

# Resultados

### Acurácia e Parâmetros do melhor modelo:

In [ ]:
#Acurácia para o conjunto de testes
print('Acurácia média para os 3 splits de teste do melhor modelo:',gs.best_score_)

print('\nParâmetros:')
for key in gs.best_params_.keys():
    print('\t',key, ': ', gs.best_params_[key])